In [ ]:
from keras.layers.embeddings import Embedding
from keras.models import Model
from keras import Input
from keras.layers import LSTM, Embedding, Dropout


class CustomModel(CustomModel):
    '''Extended CustomModel'''
    
    def buildLSTM(self, embedding, drop_rate=0.3, nb_filters=128, filter_size=3):
        self.embedding = embedding

        my_input = Input(shape=(self.paddingLength,), name= 'input')

        embedding = Embedding(input_length=self.paddingLength,
                                input_dim=self.embedding.shape[0],
                                output_dim=self.embedding.shape[1],
                                weights=[self.embedding],
                                trainable=self.trainable,
                                name='embedding')(my_input)

        embedding_dropped = Dropout(drop_rate, name='drop0')(embedding)
        lstm1 = LSTM(100, name = 'lstm1', dropout= drop_rate,
                     recurrent_dropout= drop_rate)(embedding_dropped)
        dense1 = Dense(self.nbCategories, activation = 'relu', name = 'dense1')(lstm1)
        prob = Dense(self.nbCategories, activation='softmax', name='softmax')(dense1)

        self.model = Model(my_input, prob)

        self.model.compile(
            loss='categorical_crossentropy',
            optimizer='adam',
            metrics=['accuracy']
        )

#### 2D convnet for text classification: 
Inspired from : https://machinelearningmastery.com/sequence-classification-lstm-recurrent-neural-networks-python-keras/

In [ ]:
from keras.layers.embeddings import Embedding
from keras.models import Model
from keras import Input
from keras.layers import MaxPooling1D, LSTM, Conv1D


class CustomModel(CustomModel):
    '''Extended CustomModel'''
    
    def buildLSTM_CNN(self, embedding, drop_rate=0.3, nb_filters=128, filter_size=3):
        self.embedding = embedding

        my_input = Input(shape=(self.paddingLength,), name= 'input')

        embedding = Embedding(input_length=self.paddingLength,
                                input_dim=self.embedding.shape[0],
                                output_dim=self.embedding.shape[1],
                                weights=[self.embedding],
                                trainable=self.trainable,
                                name='embedding')(my_input)

        embedding_dropped = Dropout(drop_rate, name='drop0')(embedding)
        conv1 = Conv1D(nb_filters, filter_size, activation='relu',
                       name='conv1', padding='same')(embedding_dropped)
        pooled1 = MaxPooling1D(pool_size = 2, name = 'pool1')(conv1)
        dropped1 = Dropout(drop_rate, name = 'drop1')(pooled1)
        lstm1 = LSTM(100, name = 'lstm1',
                     dropout= drop_rate, recurrent_dropout= drop_rate)(dropped1)
        prob = Dense(self.nbCategories,
                     activation='softmax', name='dense1')(lstm1)

        self.model = Model(my_input, prob)

        self.model.compile(
            loss='categorical_crossentropy',
            optimizer='adam',
            metrics=['accuracy']
        )

In [ ]:
from keras.layers.embeddings import Embedding
from keras.models import Model
from keras import Input
from keras.layers import (Input, Conv2D, Dense, Dropout,
                          MaxPooling2D, Flatten, Concatenate, Reshape)


class CustomModel(CustomModel):
    '''Extended CustomModel'''

    def buildCNN2D(self, embedding, drop_rate=0.3, nb_filters=128, filter_size=[3, 5, 8],
                   padding=PADDING):
        if np.isscalar(filter_size):
            filter_size = [3, 5, 8]
            print("WARNING: You have to enter a list for the different\
            filter sizes, we modified directly to: {}".format(filter_size))
        
        self.embedding = embedding

        my_input = Input(shape=(self.paddingLength, ), name= 'input')

        embedding = Embedding(input_dim=self.embedding.shape[0],
                               output_dim=self.embedding.shape[1],
                               weights=[self.embedding],
                               input_length=self.paddingLength,
                               trainable=self.trainable,
                               name='embedding')(my_input)
        embedding = Reshape((padding, self.embedding.shape[1], 1))(embedding)
        embedding_dropped = Dropout(drop_rate, name='drop0')(embedding)
        
        # we concatenate 3 filter sizes
        conv0 = Conv2D(nb_filters, (filter_size[0], self.embedding.shape[1]),
                       activation='relu', name='conv0', padding='valid')(embedding_dropped)
        pooled0 = MaxPooling2D(pool_size=(padding - filter_size[0] + 1, 1),
                               strides=(1, 1), padding='valid', name='pool0')(conv0)
        
        conv1 = Conv2D(nb_filters, (filter_size[1], self.embedding.shape[1]),
                       activation='relu', name='conv1', padding='valid')(embedding_dropped)
        pooled1 = MaxPooling2D(pool_size = (padding - filter_size[1] + 1, 1),
                               strides=(1, 1), padding='valid', name='pool1')(conv1)
        
        conv2 = Conv2D(nb_filters, (filter_size[2], self.embedding.shape[1]),
                       activation='relu', name='conv2', padding='valid')(embedding_dropped)
        pooled2 = MaxPooling2D(pool_size = (padding - filter_size[2] + 1, 1),
                               strides=(1, 1), padding='valid', name='pool2')(conv2)
        
        concatenated = Concatenate(axis = 1)([pooled0, pooled1, pooled2])
        flattened = keras.layers.Flatten()(concatenated)
        dropped1 = Dropout(drop_rate, name='drop1')(flattened)  
        prob = Dense(self.nbCategories, activation='softmax', name='dense2')(dropped1)

        self.model = Model(my_input, prob)

        self.model.compile(
            loss='categorical_crossentropy',
            optimizer='adam',
            metrics=['accuracy']
        )